In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_columns', 100)

In [3]:
pickle_path = '/Users/jacoblucas/group_project/Baseball/run/setup/data/analysis_dataset.pickle'
df_odds = pd.read_pickle(pickle_path)
df_allbets = pd.read_pickle('portfolio.pickle')

In [4]:
df = pd.merge(df_odds, df_allbets, on='gameno', how='right')

In [5]:
df.head()

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,v_rot,v_run_dif_game,home,h_team,h_team_run_total,h_money_line_close,h_money_line_open,h_run_line_close,h_run_line_odds_close,h_pitcher,h_rot,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open,bet_outcomes,portfolio_value,home_bet_outcomes_ml,home_portfolio_value_ml,visitor_bet_outcomes_ml,visitor_portfolio_value_ml,favorites_bet_outcomes_ml,favorites_portfolio_value_ml,underdogs_bet_outcomes_ml,underdogs_portfolio_value_ml,home_underdogs_ml_bet_outcomes_ml,home_underdogs_ml_portfolio_value_ml,visitor_favorites_ml_bet_outcomes_ml,visitor_favorites_ml_portfolio_value_ml,visitor_underdogs_ml_bet_outcomes_ml,visitor_underdogs_ml_portfolio_value_ml,home_favorites_ml_bet_outcomes_ml,home_favorites_ml_portfolio_value_ml,longshot_teams_ml_bet_outcomes_ml,longshot_teams_ml_portfolio_value_ml,home_bet_outcomes_rl,home_portfolio_value_rl,visitor_bet_outcomes_rl,visitor_portfolio_value_rl,favorites_bet_outcomes_rl,favorites_portfolio_value_rl,underdogs_bet_outcomes_rl,underdogs_portfolio_value_rl,home_favorites_rl_bet_outcomes_rl,home_favorites_rl_portfolio_value_rl,visitor_underdogs_rl_bet_outcomes_rl,visitor_underdogs_rl_portfolio_value_rl,home_underdogs_rl_bet_outcomes_rl,home_underdogs_rl_portfolio_value_rl,visitor_favorites_rl_bet_outcomes_rl,visitor_favorites_rl_portfolio_value_rl,longshot_teams_rl_bet_outcomes_rl,longshot_teams_rl_portfolio_value_rl,overs_bet_outcomes_ou,overs_portfolio_value_ou,unders_bet_outcomes_ou,unders_portfolio_value_ou,favorites_bet_outcomes_ou,favorites_portfolio_value_ou,underdogs_bet_outcomes_ou,underdogs_portfolio_value_ou
0,1.554523e+09,22296,V,WAS,5,-128.0,-110.0,-1.5,125.0,PCORBIN-L,951.0,-1.0,H,NYM,6,118.0,-110.0,1.5,-145.0,SMATZ-L,952.0,1.0,11.0,7.0,7.0,-110.0,-115.0,7.0,7.0,-110.0,-105.0,-1.000000,-100.00000,1.180000,-34929.75391,-1.00,-31828.013048,-1.000000,-31836.266958,1.18,-34921.5,1.18,-12898.5,-1.0,-9805.013048,0.00,-22023.0,0.000000,-22031.25391,0.0,-8677.0,1.45,-75690.636839,-1.00,385414.669705,-1.0,-332976.967134,1.45,642701.0,0.0,-239316.636839,0.00,479075.0,1.45,163626.0,-1.0,-93660.330295,0.0,0.0,1.10,-33794.909687,-1.0,-28692.181412,-1.0,-58437.091099,1.10,-4050.0
1,1.554523e+09,22297,V,CIN,5,101.0,115.0,1.5,-205.0,TROARK-R,953.0,-1.0,H,PIT,6,-111.0,-135.0,-1.5,172.0,TWILLIAMS-R,954.0,1.0,11.0,8.0,7.0,-105.0,-115.0,8.0,7.0,-115.0,-105.0,0.900901,-9.90991,0.900901,-34839.66382,-1.00,-31928.013048,0.900901,-31746.176868,-1.00,-35021.5,0.00,-12898.5,0.0,-9805.013048,-1.00,-22123.0,0.900901,-21941.16382,0.0,-8677.0,-1.00,-75790.636839,2.05,385619.669705,-1.0,-333076.967134,2.05,642906.0,-1.0,-239416.636839,2.05,479280.0,0.00,163626.0,0.0,-93660.330295,0.0,0.0,1.05,-33689.909687,-1.0,-28792.181412,-1.0,-58537.091099,1.05,-3945.0
2,1.554523e+09,22298,V,SD,6,110.0,120.0,1.5,-205.0,CPADDACK-R,955.0,2.0,H,STL,4,-120.0,-140.0,-1.5,172.0,MWACHA-R,956.0,-2.0,10.0,7.5,7.5,-105.0,-115.0,7.5,7.5,-115.0,-105.0,-1.000000,-109.90991,-1.000000,-34939.66382,1.10,-31818.013048,-1.000000,-31846.176868,1.10,-34911.5,0.00,-12898.5,0.0,-9805.013048,1.10,-22013.0,-1.000000,-22041.16382,0.0,-8677.0,-1.00,-75890.636839,2.05,385824.669705,-1.0,-333176.967134,2.05,643111.0,-1.0,-239516.636839,2.05,479485.0,0.00,163626.0,0.0,-93660.330295,0.0,0.0,1.05,-33584.909687,-1.0,-28892.181412,-1.0,-58637.091099,1.05,-3840.0
3,1.554523e+09,22299,V,CHC,14,119.0,100.0,1.5,-180.0,CHAMELS-L,957.0,6.0,H,MIL,8,-129.0,-120.0,-1.5,155.0,CBURNES-R,958.0,-6.0,22.0,8.5,8.5,-105.0,-120.0,8.5,8.5,-115.0,100.0,-1.000000,-209.90991,-1.000000,-35039.66382,1.19,-31699.013048,-1.000000,-31946.176868,1.19,-34792.5,0.00,-12898.5,0.0,-9805.013048,1.19,-21894.0,-1.000000,-22141.16382,0.0,-8677.0,-1.00,-75990.636839,1.80,386004.669705,-1.0,-333276.967134,1.80,643291.0,-1.0,-239616.636839,1.80,479665.0,0.00,1636

In [6]:
v_index_ls = []
h_index_ls = []
for index, row in df.iterrows():
    v_ml_open = row['v_money_line_open']
    v_ml_close = row['v_money_line_close']
    h_ml_open = row['h_money_line_open']
    h_ml_close = row['h_money_line_close']
    
    if v_ml_close < v_ml_open:
        v_index_ls.append(index)
    
    if h_ml_close < h_ml_open:
        h_index_ls.append(index)
v_pub_fav = df.iloc[v_index_ls, :]
h_pub_fav = df.iloc[h_index_ls, :]

In [7]:
v_pub_fav.head(20)

,date,gameno,visitor,v_team,v_team_run_total,v_money_line_close,v_money_line_open,v_run_line_close,v_run_line_odds_close,v_pitcher,v_rot,v_run_dif_game,home,h_team,h_team_run_total,h_money_line_close,h_money_line_open,h_run_line_close,h_run_line_odds_close,h_pitcher,h_rot,h_run_dif_game,total_runs_game,over_line_close,over_line_open,over_odds_close,over_odds_open,under_line_close,under_line_open,under_odds_close,under_odds_open,bet_outcomes,portfolio_value,home_bet_outcomes_ml,home_portfolio_value_ml,visitor_bet_outcomes_ml,visitor_portfolio_value_ml,favorites_bet_outcomes_ml,favorites_portfolio_value_ml,underdogs_bet_outcomes_ml,underdogs_portfolio_value_ml,home_underdogs_ml_bet_outcomes_ml,home_underdogs_ml_portfolio_value_ml,visitor_favorites_ml_bet_outcomes_ml,visitor_favorites_ml_portfolio_value_ml,visitor_underdogs_ml_bet_outcomes_ml,visitor_underdogs_ml_portfolio_value_ml,home_favorites_ml_bet_outcomes_ml,home_favorites_ml_portfolio_value_ml,longshot_teams_ml_bet_outcomes_ml,longshot_teams_ml_portfolio_value_ml,home_bet_outcomes_rl,home_portfolio_value_rl,visitor_bet_outcomes_rl,visitor_portfolio_value_rl,favorites_bet_outcomes_rl,favorites_portfolio_value_rl,underdogs_bet_outcomes_rl,underdogs_portfolio_value_rl,home_favorites_rl_bet_outcomes_rl,home_favorites_rl_portfolio_value_rl,visitor_underdogs_rl_bet_outcomes_rl,visitor_underdogs_rl_portfolio_value_rl,home_underdogs_rl_bet_outcomes_rl,home_underdogs_rl_portfolio_value_rl,visitor_favorites_rl_bet_outcomes_rl,visitor_favorites_rl_portfolio_value_rl,longshot_teams_rl_bet_outcomes_rl,longshot_teams_rl_portfolio_value_rl,overs_bet_outcomes_ou,overs_portfolio_value_ou,unders_bet_outcomes_ou,unders_portfolio_value_ou,favorites_bet_outcomes_ou,favorites_portfolio_value_ou,underdogs_bet_outcomes_ou,underdogs_portfolio_value_ou
0,1.554523e+09,22296,V,WAS,5,-128.0,-110.0,-1.5,125.0,PCORBIN-L,951.0,-1.0,H,NYM,6,118.0,-110.0,1.5,-145.0,SMATZ-L,952.0,1.0,11.0,7.0,7.0,-110.0,-115.0,7.0,7.0,-110.0,-105.0,-1.000000,-100.000000,1.180000,-34929.753910,-1.000000,-31828.013048,-1.000000,-31836.266958,1.18,-34921.5,1.18,-12898.5,-1.000000,-9805.013048,0.0,-22023.0,0.000000,-22031.253910,0.0,-8677.0,1.450000,-75690.636839,-1.000000,385414.669705,-1.000000,-332976.967134,1.45,642701.0,0.000000,-239316.636839,0.00,479075.0,1.45,163626.0,-1.000000,-93660.330295,0.0,0.0,1.10,-33794.909687,-1.000000,-28692.181412,-1.000000,-58437.091099,1.10,-4050.0
1,1.554523e+09,22297,V,CIN,5,101.0,115.0,1.5,-205.0,TROARK-R,953.0,-1.0,H,PIT,6,-111.0,-135.0,-1.5,172.0,TWILLIAMS-R,954.0,1.0,11.0,8.0,7.0,-105.0,-115.0,8.0,7.0,-115.0,-105.0,0.900901,-9.909910,0.900901,-34839.663820,-1.000000,-31928.013048,0.900901,-31746.176868,-1.00,-35021.5,0.00,-12898.5,0.000000,-9805.013048,-1.0,-22123.0,0.900901,-21941.163820,0.0,-8677.0,-1.000000,-75790.636839,2.050000,385619.669705,-1.000000,-333076.967134,2.05,642906.0,-1.000000,-239416.636839,2.05,479280.0,0.00,163626.0,0.000000,-93660.330295,0.0,0.0,1.05,-33689.909687,-1.000000,-28792.181412,-1.000000,-58537.091099,1.05,-3945.0
2,1.554523e+09,22298,V,SD,6,110.0,120.0,1.5,-205.0,CPADDACK-R,955.0,2.0,H,STL,4,-120.0,-140.0,-1.5,172.0,MWACHA-R,956.0,-2.0,10.0,7.5,7.5,-105.0,-115.0,7.5,7.5,-115.0,-105.0,-1.000000,-109.909910,-1.000000,-34939.663820,1.100000,-31818.013048,-1.000000,-31846.176868,1.10,-34911.5,0.00,-12898.5,0.000000,-9805.013048,1.1,-22013.0,-1.000000,-22041.163820,0.0,-8677.0,-1.000000,-75890.636839,2.050000,385824.669705,-1.000000,-333176.967134,2.05,643111.0,-1.000000,-239516.636839,2.05,479485.0,0.00,163626.0,0.000000,-93660.330295,0.0,0.0,1.05,-33584.909687,-1.000000,-28892.181412,-1.000000,-58637.091099,1.05,-3840.0
11,1.554523e+09,22307,V,OAK,0,125.0,144.0,1.5,-154.0,ABROOKS-R,973.0,-6.0,H,HOU,6,-135.0,-165.0,-1.5,136.0,WMILEY-L,974.0,6.0,6.0,9.5,8.5,100.0,-101.0,9.5,8.5,-120.0,-119.0,0.740741,1.940221,0.740741,-35169.576141,-1.000000,-31691.250596,0.740741,-31734.326737,-1.00,-35126.5,0.00,-13098.5,0.000000,-9663.250596,-1.0,-22028.0,0.740741,-2207

In [9]:
h_against_pub_profit = v_pub_fav['home_favorites_ml_bet_outcomes_ml'].sum() + v_pub_fav['home_underdogs_ml_bet_outcomes_ml'].sum()
v_against_pub_profit = h_pub_fav['visitor_favorites_ml_bet_outcomes_ml'].sum() + h_pub_fav['visitor_underdogs_ml_bet_outcomes_ml'].sum()
total = h_against_pub_profit + v_against_pub_profit
total

3.638178461221692

In [35]:
# create dfs based on different metrics
heavy_home_fav = df.loc[(df['h_money_line_close'] < (-200))]
heavy_vis_fav = df.loc[(df['v_money_line_close'] < (-200))]
home_fav = df[df['h_money_line_close'].between(-200, 0, inclusive=True)]
vis_fav = df[df['v_money_line_close'].between(-200, 0, inclusive=True)]

In [41]:
# profitability of heavy home favorites
heavy_home_fav_winners = heavy_home_fav.loc[heavy_home_fav['h_run_dif_game'] > 0]
h_percent_win = len(heavy_home_fav_winners) / len(heavy_home_fav)
profit = heavy_home_fav['home_favorites_ml_bet_outcomes_ml'].sum()
h_percent_win, profit

(0.7047461368653422, -2.3076273651722605)

In [42]:
# profitability of normal home favorites
home_fav_winners = home_fav.loc[home_fav['h_run_dif_game'] > 0]
h_percent_win = len(home_fav_winners) / len(home_fav)
profit = home_fav['home_favorites_ml_bet_outcomes_ml'].sum()
h_percent_win, profit

(0.5614160814103483, -383.8112701050886)

In [43]:
# profitability of heavy vis favorites
heavy_vis_fav_winners = heavy_vis_fav.loc[heavy_vis_fav['v_run_dif_game'] > 0]
v_percent_win = len(heavy_vis_fav_winners) / len(heavy_vis_fav)
profit = heavy_vis_fav['visitor_favorites_ml_bet_outcomes_ml'].sum()
v_percent_win, profit

(0.6750700280112045, -12.632557564125193)

In [44]:
# profitability of normal vis favorites
vis_fav_winners = vis_fav.loc[vis_fav['v_run_dif_game'] > 0]
v_percent_win = len(vis_fav_winners) / len(vis_fav)
profit = vis_fav['visitor_favorites_ml_bet_outcomes_ml'].sum()
v_percent_win, profit

(0.5452621093263603, -159.97104644470477)